In [1]:
# choose language (affect dataset)
RU, EN = "russian", "english"
engine = RU

In [5]:
import spacy
import nltk
import pandas as pd
from pymystem3 import Mystem
from string import punctuation

class EnLemmaTokenizer:
    def  __init__(self):
        self.nlp = spacy.load("en")
    def __call__(self, text):
        return [
            token.lemma_ for token in self.nlp(text)
            if not (token.is_stop or token.is_punct or token.is_digit)
        ]

class RuLemmaTokenizer:
    def  __init__(self):
        nltk.download("stopwords")
        self.stopwords = nltk.corpus.stopwords.words("russian")
        self.lemma = Mystem().lemmatize
    def __call__(self, text):
        return [
            token for token in self.lemma(text)
            if token not in self.stopwords and token not in punctuation
        ]

kp_dataset = "https://raw.githubusercontent.com/madmaxeatfax/fellini/master/dataset/kp250.csv"
imdb_dataset = "https://raw.githubusercontent.com/madmaxeatfax/fellini/master/dataset/imdb250.csv"

lemma_tokenizer = EnLemmaTokenizer() if engine == EN else RuLemmaTokenizer()
bunch = pd.read_csv(imdb_dataset if engine == EN else kp_dataset)
bunch.fillna("", inplace=True)

[nltk_data] Downloading package stopwords to /Users/madma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start_time = time.time()

vec = TfidfVectorizer(max_features=50_000, tokenizer=lemma_tokenizer)
data = \
    bunch.Title + " " + bunch.Crew + " " + bunch.Plot + " " + \
    bunch.Tags + " " + bunch.Country + " " + bunch.Reviews

# print([i for i, x in enumerate(data.isna()) if x == True])

features = vec.fit_transform(data)

print(f'Training time {time.time() - start_time}s, features number = {len(vec.get_feature_names())}')

Training time 62.92034935951233s, features number = 34291
['\t', '\t ', '\n', ' ', ' \t', ' \t\t ', ' \t ', ' \t- ', ' \t«', ' \n', '  ', '  \t', '   ', '   -', '   «', '  (', '  *', '  *** ', '  -', '  - ', '  «', '  « — ', '  «- ', '  •\t', ' % ', ' %) ', ' &', ' & ', " ' ", ' (']


In [7]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [28]:
# input your search query
QUERY = "фильм про бухгалтера, который попал в тюрьму"

start_time = time.time()
query_vector = vec.transform([QUERY])
print(f'Query = {QUERY}\n')

distances, neighbors = knn.kneighbors(query_vector, n_neighbors=5, return_distance=True)
    
for dist, neighbor_idx in zip(distances[0], neighbors[0]):
    print(*[
        bunch.Title[neighbor_idx],
        f'Distance = {dist}  Neighbor idx = {neighbor_idx}',
        bunch.Plot[neighbor_idx][:200], 
        bunch.Crew[neighbor_idx],
        "-"*200
    ], sep="\n")

print(f'\nQuery execution time - {time.time() - start_time:.5f} sec')


Query = фильм про бухгалтера, который попал в тюрьму

Побег из Шоушенка 
Distance = 0.34035973064960356  Neighbor idx = 0
Бухгалтер Энди Дюфрейн обвинён в убийстве собственной жены и её любовника. Оказавшись в тюрьме под названием Шоушенк, он сталкивается с жестокостью и беззаконием, царящими по обе стороны решётки. Кажд
 Фрэнк Дарабонт,  Фрэнк Дарабонт,  Стивен Кинг, Тим Роббинс, Морган Фриман, Боб Гантон, Уильям Сэдлер, Клэнси Браун
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Зеленая миля 
Distance = 0.34283875151600596  Neighbor idx = 1
Пол Эджкомб — начальник блока смертников в тюрьме «Холодная гора», каждый из узников которого однажды проходит «зеленую милю» по пути к месту казни. Пол повидал много заключённых и надзирателей за вре
 Фрэнк Дарабонт,  Фрэнк Дарабонт,  Стивен Кинг, Том Хэнкс, Дэвид Морс, Бонни Хант, Майкл Кларк Д